<a href="https://colab.research.google.com/github/tanmana5/mangrove-diaries/blob/main/Python_Project_Mangrove_Mapping_with_Rasterio_and_Area_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#
# Python Project to Map Mangrove Areas in India using Rasterio
#
# This script demonstrates a raster-based approach for mapping mangroves.
# It uses the Normalized Difference Vegetation Index (NDVI) as a method
# to identify dense vegetation from satellite imagery. It also now
# calculates and prints the total area of the mapped mangroves.
#
# Note: This code is a template. You will need to acquire a satellite image
# (e.g., from Sentinel-2 or Landsat) with visible and near-infrared bands
# for an area in India. Place the image file in the 'data' folder.
#

import rasterio
import numpy as np
import matplotlib.pyplot as plt
import os

def create_project_structure(project_name="mangrove_project"):
    """
    Creates a basic directory structure for the project.
    """
    try:
        os.makedirs(f"{project_name}/data", exist_ok=True)
        os.makedirs(f"{project_name}/output", exist_ok=True)
        print(f"Project structure created for '{project_name}'.")
        print("Please place your satellite image file (e.g., 'satellite_image.tif') in the 'data' folder.")
    except OSError as e:
        print(f"Error creating directories: {e}")

def calculate_ndvi(red_band, nir_band):
    """
    Calculates the Normalized Difference Vegetation Index (NDVI).

    NDVI = (NIR - Red) / (NIR + Red)

    Args:
        red_band (np.array): A numpy array of the red band pixel values.
        nir_band (np.array): A numpy array of the near-infrared band pixel values.

    Returns:
        np.array: A numpy array of the calculated NDVI values.
    """
    # Avoid division by zero by converting zero values to a small number
    with np.errstate(divide='ignore', invalid='ignore'):
        ndvi = (nir_band.astype(float) - red_band.astype(float)) / (nir_band.astype(float) + red_band.astype(float))
    return ndvi

def calculate_area(classified_raster, transform):
    """
    Calculates the total area of the classified mangroves.

    Args:
        classified_raster (np.array): A numpy array where 1 represents mangrove pixels.
        transform (affine.Affine): The affine transform from the raster metadata,
                                   which contains the pixel size.

    Returns:
        float: The total area in square meters.
    """
    # Get the pixel width and height from the affine transform.
    # The transform is a 2D array, so we access the relevant values.
    pixel_width = abs(transform[0])
    pixel_height = abs(transform[4])
    pixel_area = pixel_width * pixel_height

    # Count the number of pixels classified as mangrove (value of 1)
    mangrove_pixel_count = np.sum(classified_raster == 1)

    # Calculate the total area in square meters
    total_area_sqm = mangrove_pixel_count * pixel_area

    return total_area_sqm

def main():
    """
    Main function to perform the mangrove mapping workflow using raster data.
    """
    project_name = "mangrove_project"
    create_project_structure(project_name)

    # Define file paths
    # Replace 'satellite_image.tif' with the name of your downloaded raster file.
    image_path = f"{project_name}/data/satellite_image.tif"
    output_path = f"{project_name}/output/india_mangroves_raster.tif"
    output_map_path = f"{project_name}/output/india_mangroves_map.png"

    # Check if the satellite image file exists
    if not os.path.exists(image_path):
        print("\nRequired satellite image file not found!")
        print(f"Please place your satellite image, for example 'satellite_image.tif', in the '{project_name}/data' directory.")
        return

    try:
        # Step 1: Open the satellite image using rasterio
        print("\nLoading satellite image...")
        with rasterio.open(image_path) as src:
            # We assume a common band order for Sentinel-2 or Landsat-8.
            # Red band is typically Band 4, and Near-Infrared (NIR) is Band 8.
            # You may need to adjust these band indices based on your specific data.
            # Rasterio uses 1-based indexing for bands.
            band_red = src.read(4)
            band_nir = src.read(8)
            profile = src.profile
            transform = src.transform

            # Update the profile for the output raster (new dtype, single band)
            profile.update(dtype=rasterio.float32, count=1, driver='GTiff')

        # Step 2: Calculate the NDVI
        print("Calculating NDVI...")
        ndvi_array = calculate_ndvi(band_red, band_nir)

        # Step 3: Classify mangroves based on a threshold
        # Mangroves are a form of dense, healthy vegetation, so they will have
        # high NDVI values. A common threshold is > 0.4.
        print("Classifying mangroves based on NDVI threshold...")
        mangrove_mask = np.where(ndvi_array > 0.4, 1, 0)

        # Step 4: Save the new mangrove mask as a GeoTIFF file
        print(f"Saving the mangrove mask to '{output_path}'...")
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(mangrove_mask.astype(rasterio.float32), 1)

        # Step 5: Calculate the total area
        print("Calculating total mangrove area...")
        total_area_sqm = calculate_area(mangrove_mask, transform)
        total_area_sqkm = total_area_sqm / 1_000_000  # Convert to square kilometers
        print(f"Total calculated mangrove area: {total_area_sqkm:.2f} sq km")

        # Step 6: Visualize the results
        print("Generating and saving the visualization...")
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

        # False-color composite for context (NIR, Red, Green)
        # Assuming bands 8, 4, 3 are NIR, Red, Green for Sentinel-2
        false_color_composite = np.stack([band_nir, band_red, src.read(3)], axis=-1)
        ax1.imshow(false_color_composite / np.max(false_color_composite))
        ax1.set_title("Original Satellite Image (False-Color Composite)", fontsize=16)
        ax1.axis('off')

        # Plot the mangrove mask
        im = ax2.imshow(mangrove_mask, cmap='YlGn', vmin=0, vmax=1)
        ax2.set_title("Mangrove Classification", fontsize=16)
        ax2.axis('off')

        # Add a legend or color bar
        cbar = fig.colorbar(im, ax=ax2, fraction=0.046, pad=0.04)
        cbar.ax.set_yticklabels(['Non-Mangrove', 'Mangrove'])

        plt.savefig(output_map_path, bbox_inches='tight')
        plt.show()

        print(f"\nProcessing complete! Classified map saved to '{output_path}' and visualization saved to '{output_map_path}'.")

    except rasterio.errors.RasterioIOError as e:
        print(f"Rasterio error: Could not open or read the image file. Ensure the file is a valid raster and the path is correct. Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    main()